In [20]:
import tensorflow as tf
import numpy as np
import os
import time

In [21]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 
print(tf.config.list_physical_devices())
path_to_file = "A:/Neural Network/IliadExodus.txt"

['/device:CPU:0', '/device:GPU:0']
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [22]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters and {text.count(" ") + text.count(chr(13))} words')
vocab = sorted(set(text))

len(vocab)

Length of text: 836287 characters and 135435 words


222

In [23]:
#make some basic functions, so tensor flow can read our texts
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [24]:
#make different small sequences
seq_length = 100
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [25]:
#chop up text into small bits for easier training.
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [26]:
#split the dataset in chuncks acceptable by tensorflow
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
split_input_target(list("Tensorflow"))
dataset = sequences.map(split_input_target)

In [27]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [28]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [29]:
class TextModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return {'loss': loss}

In [30]:
model = TextModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 223) # (batch_size, sequence_length, vocab_size)


In [32]:
model.summary()

Model: "text_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  57088     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  228575    
                                                                 
Total params: 4,223,967
Trainable params: 4,223,967
Non-trainable params: 0
_________________________________________________________________


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 223)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.4077315, shape=(), dtype=float32)


In [34]:
tf.exp(example_batch_mean_loss).numpy()
model.compile(optimizer='adam', loss=loss)
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
129/129 [==============================] - 19s 132ms/step - loss: 3.4659
Epoch 2/20
129/129 [==============================] - 19s 132ms/step - loss: 2.4391
Epoch 3/20
129/129 [==============================] - 20s 140ms/step - loss: 2.1713
Epoch 4/20
129/129 [==============================] - 19s 135ms/step - loss: 1.9832
Epoch 5/20
129/129 [==============================] - 19s 138ms/step - loss: 1.8012
Epoch 6/20
129/129 [==============================] - 19s 137ms/step - loss: 1.6458
Epoch 7/20
129/129 [==============================] - 18s 136ms/step - loss: 1.5219
Epoch 8/20
129/129 [==============================] - 19s 139ms/step - loss: 1.4175
Epoch 9/20
129/129 [==============================] - 18s 134ms/step - loss: 1.3271
Epoch 10/20
129/129 [==============================] - 18s 134ms/step - loss: 1.2447
Epoch 11/20
129/129 [==============================] - 18s 134ms/step - loss: 1.1700
Epoch 12/20
129/129 [==============================] - 18s 135ms/step - lo

In [36]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [38]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant(['οἰ δε Ἀχιλῆοι καὶ τε Ἑβραίων'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

οἰ δε Ἀχιλῆοι καὶ τε Ἑβραίων παρὰ τοῦ ἑνὸς αὐτῆς, καὶ ἔσται Ἀχιλῆα προχείπονται ὁ ταῦρος καὶ ὅ κε σὺ καὶ πάχας ἐπὶ δὲ τῆς κεφαλῇ, καὶ ἐπίστευσαν ὥστα ἐπὶ τῶν στύλων ἡμῶν, καὶ εἶπε Μωυσῆς·
τί λοπτοῦ·
Τὸρ δμῳθήτηρ Κύριος ἡμέρᾳ τῇ ἑβδόμῃ·
καὶ ἰδοὺ οὐκ ἐξ ὑπὸ τὴν ἁμὸν κατὰ καλίν.
καθότι οὐδὲ ἔτι μᾶλλον ὅτι ἐπὶ τῆς γῆς, καὶ προσήγαγον αὐτοὺς ὑπὸ τὸ ὕδωρ, καὶ ἔσται, ὅτι μὴ ραὸς τῆς Φαραὼ καὶ Ἀλυονῆφαι Φαραώ·
τοῦτό μόματος τρόμους ἔκουσον πυρὰ τοῦ Θεοῦ καὶ ἀποθάνῃ, λαβλητὰ τὴν παρθνόν σου καὶ ἡμίσους τὸ ὕψος·
αὐτὸν δὲ οἱ ἄρχοντες τοῖς ζεοῖς.
καὶ ποιήσεις τὰ ἔργα Ἰσραὴλ τῷ στύλῳ; τὰ δὲ βραχύοντες ἔνυσαν ἀπέλῃκε, καὶ τρεῖς κρατῆρ αὐτῆς καὶ λίθους τοῦ Ἰσραήλ, πᾶς εἰσακάριες αὐτήν.
καὶ πεντήκοντα πήχεων τὸ μέσον τοῦ ἐνιαυτοῦ λαῷ·
ἐν γὰρ χειρὶ κρατύσω ὑμᾶς καὶ πάσῃς τῆς φωνῆς μί τις πρὸς ὄψομαι, εἰς ὄφρα πάντα τὸν λαόν μου τὰ ἔργεν ὑμῖν αὐτῷ·
τά γὰρ καθάπερ χθές, καὶ οὐ δυνήσονται ὅτι νότος καὶ πάντες οἱ Αἰγύπτιοι Μωυσῆς μέγα τῆς θῆκεν ὑμῖν μαχαῖρας ἐπὶ σέ, γούνυκτο δὲ ταῦτα μαχήσεται αὐτὰ καὶ υἱο